In [1]:
# -*- coding: utf-8 -*-

""" Auto Encoder Example.
Using an auto encoder on MNIST handwritten digits.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
"""
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

df = pd.read_csv("../dat/diabetic.txt", header=None);


In [26]:
# data.shape #(1151, 20)
#type(data) #pandas.core.frame.DataFrame
train = df.head(1000)
train_x = train.T.head(18)
train_y = train.T.tail(1)
test = df.tail(151)
test_x = test.T.head(150)

In [36]:
# Parameters
learning_rate = 0.1
training_epochs = 50
# batch_size = 256
batch_size = 19
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 19 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input = 1000 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(n_input / batch_size) #(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = train_x, train_y# mnist.train.next_batch(batch_size) # 256x1, 256x1
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))

    print("Optimization Finished!")

    # Applying encode and decode over test set
    encode_decode = sess.run(
        y_pred, feed_dict={X: test_x.T}) #{X: mnist.test.images[:examples_to_show]})
#     # Compare original images with their reconstructions
#     f, a = plt.subplots(2, 10, figsize=(10, 2))
#     for i in range(examples_to_show):
#         a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
#         a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
#     f.show()
#     plt.draw()
#     plt.waitforbuttonpress()

Epoch: 0001 cost= 968.337768555
Epoch: 0002 cost= 960.324096680
Epoch: 0003 cost= 958.272216797
Epoch: 0004 cost= 957.918212891
Epoch: 0005 cost= 957.895996094
Epoch: 0006 cost= 957.898437500
Epoch: 0007 cost= 957.880249023
Epoch: 0008 cost= 957.876220703
Epoch: 0009 cost= 957.874572754
Epoch: 0010 cost= 957.874023438
Epoch: 0011 cost= 957.873779297
Epoch: 0012 cost= 957.873535156
Epoch: 0013 cost= 957.873107910
Epoch: 0014 cost= 957.872680664
Epoch: 0015 cost= 957.872680664
Epoch: 0016 cost= 957.872436523
Epoch: 0017 cost= 957.872436523
Epoch: 0018 cost= 957.872192383
Epoch: 0019 cost= 957.872192383
Epoch: 0020 cost= 957.872192383
Epoch: 0021 cost= 957.871765137
Epoch: 0022 cost= 957.871643066
Epoch: 0023 cost= 957.871582031
Epoch: 0024 cost= 957.871215820
Epoch: 0025 cost= 957.871215820
Epoch: 0026 cost= 957.871215820
Epoch: 0027 cost= 957.871215820
Epoch: 0028 cost= 957.870971680
Epoch: 0029 cost= 957.870910645
Epoch: 0030 cost= 957.870910645
Epoch: 0031 cost= 957.870666504
Epoch: 0

ValueError: Cannot feed value of shape (151, 20) for Tensor u'Placeholder_15:0', which has shape '(?, 1000)'

In [35]:
test_x

,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
2,35.000000,79.000000,5.000000,8.000000,6.000000,18.000000,29.000000,35.000000,92.000000,43.000000,...,53.000000,53.000000,12.000000,35.000000,16.000000,34.000000,49.000000,49.000000,39.000000,7.000000
3,31.000000,76.000000,4.000000,8.000000,6.000000,17.000000,28.000000,35.000000,84.000000,43.000000,...,53.000000,53.000000,12.000000,34.000000,16.000000,34.000000,49.000000,48.000000,36.000000,7.000000
4,28.000000,74.000000,4.000000,8.000000,6.000000,17.000000,26.000000,34.000000,78.000000,43.000000,...,52.000000,51.000000,10.000000,33.000000,15.000000,34.000000,49.000000,48.000000,29.000000,7.000000
5,25.000000,72.000000,4.000000,8.000000,6.000000,17.000000,22.000000,32.000000,71.000000,39.000000,...,51.000000,47.000000,9.000000,33.000000,14.000000,33.000000,49.000000,45.000000,23.000000,7.000000
6,20.000000,69.000000,4.000000,5.000000,5.000000,14.000000,17.000000,30.000000,60.000000,36.000000,...,51.000000,44.000000,7.000000,33.000000,12.000000,31.000000,45.000000,43.000000,13.000000,7.000000
7,16.000000,50.000000,3.000000,4.000000,4.000000,14.000000,11.000000,22.000000,43.000000,28.000000,...,41.000000,32.000000,6.000000,26.000000,8.000000,24.000000,37.000000,33.000000,7.000000,5.000000
8,52.805279,61.559348,169.099820,87.943106,276.491407,59.655889,38.812879,16.740482,20.384375,69.952496,...,8.632010,25.283836,243.066702,2.579859,158.177307,6.071765,63.197145,30.461898,40.525739,69.423565
9,27.866246,28.959444,78.742242,28.050493,130.357061,22.852706,27.759006,8.434313,10.683792,40.476639,...,3.852405,8.337976,26.650463,0.001552,84.865487,0.937472,27.377668,13.966980,12.604947,7.031843
